In [232]:
from dataclasses import dataclass
from jsonschema import validate

import logging
import json
import requests

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [233]:
@dataclass
class Developer:
    login: str
    site_admin: bool
    repos_url: str
    node_id: str
    
expected_schema = {
    "required": ["repos_url"],
    "properties": {
        "login": {"type": "string"},
        "site_admin": {"type": "boolean"},
        "repos_url": {"type": "string"},
        "node_id": {"type": "string"},
    }
}


def get_filtered_dict_from_keys(selected_keys: List[str], data: dict) -> dict:
    return dict(
        zip(selected_keys, [user_data.get(value) for value in selected_keys])
    )

def main() -> None:
    r = requests.get("https://api.github.com/users/henmachado")

    if r.status_code == 200:
        try:
            user_data = r.json()

            logger.info('Validating received JSON...........1/4')
            validate(user_data, expected_schema)
            
        except ValidationError as ve:
            logger.info("------------Invalid JSON--------------")
            logger.info(repr(ve))
            raise

        logger.info('Getting selected keys..............2/4')
        selected_keys = ['login', 'site_admin', 'repos_url', "node_id"]
        filtered_user_data = get_filtered_dict_from_keys(selected_keys, user_data)
        logger.info(json.dumps(filtered_user_data, indent=4))
        logger.info('JSON Parsed Successfully...........3/4')

        developer = Developer(**filtered_user_data)
        logger.info('JSON Decoded to Developer object...4/4')
        logger.info('--------------------------------------')
        
        repos = requests.get(developer.repos_url).json()
        python_repos = len(
            [repo.get("language") for repo in repos if repo.get("language") == "Python"]
        )
        
        logger.info(f'User {developer.login} has {str(python_repos)} Python public projects')

    else:
        logger.info(f'Request Failed: {str(r.status_code)}')
        logger.info(r.text)
    

if __name__ == "__main__":
    main()



INFO:__main__:Validating received JSON...........1/4
INFO:__main__:Getting selected keys..............2/4
INFO:__main__:{
    "login": "Henmachado",
    "site_admin": false,
    "repos_url": "https://api.github.com/users/Henmachado/repos",
    "node_id": "MDQ6VXNlcjQxNDU2OTcy"
}
INFO:__main__:JSON Parsed Successfully...........3/4
INFO:__main__:JSON Decoded to Developer object...4/4
INFO:__main__:--------------------------------------
INFO:__main__:User Henmachado has 2 Python public projects
